In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df_social = pd.read_csv('../Data/Preprocessed_data/SocialMediaReviews.csv')
df_social.drop(columns=['ViewCount'],inplace=True)
df_social.dropna(inplace=True)
print(len(df_social))


60285


In [3]:
documents = df_social['Review']

Preprocessing

In [11]:
# from multiprocessing import Pool
from nltk import word_tokenize

documents = df_social['Review']
stopwords = set(open('./vietnamese-stopwords.txt', encoding='utf-8').read().split('\n')[:-1])
puct_set = set([c for c in '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~'])

def generateBigram(review):
    words = review.split()
    if len(words)==1:
        return ''
    bigrams = [words[i]+'_'+words[i+1] for i in range(0,len(words)-1)]
    return ' '.join(bigrams)

def removeRedundant(text,redundantSet):
    words = text.split()
    for i in range(0,len(words)):
        if words[i].count('_') == 0 and (words[i] in redundantSet or words[i].isdigit()):
            words[i] = ''
        else:
            sub_words = words[i].split('_')
            if any(w in redundantSet or w.isdigit() for w in sub_words):
                words[i] = ''
    words = [w for w in words if w != '']
    words = ' '.join(words)
    return words


def preprocessing(text):
    text = ' '.join(word_tokenize(text))
    text = text.lower()
    text = ' '.join(text.split())
    text = text + generateBigram(text)
    text = removeRedundant(text,puct_set | stopwords)
    return text

clean_documents =  documents.apply(preprocessing)

In [4]:

text_preprocessed.preprocessing_text('viết cầm 14prm vs m chẳng 😆 đàn yếu đuối thế bài_viết vs_m m_chẳng 😆_đàn yếu_đuối')

'viết cầm pro max với mình chẳng tích_cực đàn yếu_đuối thế bài_viết vs_m m_chẳng tích_cực _đàn yếu_đuối'

In [7]:
from text_preprocessed import preprocessing_text

try:
    cleaned_documents = documents.apply(preprocessing_text)
except Exception as e:
    print(e)

'NoneType' object is not subscriptable


In [18]:
documents[155:170]

156                                được cái mã bên ngoài
157                                    cái mã ngoài   đó
158                             vẫn chưa hiểu tích_cực  
159    bên ngành ông tô người_ta gọi facelift nâng_cấ...
160    mấy thằng member tinhte từ_tục thấy chia_sẻ th...
161    giàu bằng mồ_hôi của mình thì khoe mạnh lên ae...
162    mình không ganh_ghét hay đố_kị những người già...
163    chỉ chửi nhưng lại không ganh_ghét đố_kỵ   hay...
164         chấp làm gì   nó chắc đang iphone nên cay_cú
165    giàu chân_chính mà khoe thì đáng ngưởng mộ chứ...
166    nhiều người mua iphone làm động mồ động mả bọn...
167                             gato kìa lêu lêu   cay à
168                             bớt sân si và đố_kỵ đi  
169    ranh_con   có tí tiền đã khoe_khoang   muốn là...
170    Ranh con, có tí tiền đã khoe khoang, muốn làm ...
Name: Review, dtype: object

In [27]:
# from text_preprocessed import convert_emoji,remove_abbreviations,remove_replicated
# from pyvi import ViTokenizer
# text = "Có mỗi iphone  mà cả tuần này tinh tế xào nấu kho hâm lại chắc dùng 2 tháng mới hết 🤣"
# text_pre = text.lower()
# import re
# # Loại bỏ url
# text_pre = re.sub(r'http\S+', '', text_pre)

# # Loại bỏ tên miền
# text_pre = re.sub(r"[^\s]*\.(com|vn|net)", '', text_pre)

# # Loại bỏ hashtag
# text_pre = ' '.join(re.sub("(#[A-Za-z0-9]+)", " ", text_pre).split())

# # Loại bỏ user mentions @
# text_pre = ' '.join(re.sub("(@[A-Za-z0-9]+)", " ", text_pre).split())

# # Loại bỏ dấu xuống dòng \r, \n và tab \t
# text_pre = text_pre.replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')

# # Biến đổi emoji
# # text_pre = convert_emoji(text_pre)

# # Loại bỏ chữ bị lặp
# text_pre = remove_replicated(text_pre)

# # Loại bỏ chữ số
# text_pre = re.sub("\d+", " ", text_pre)

# # Loại bỏ từ viết tắt và viết sai
# text_pre = remove_abbreviations(text_pre)

# # Tokenize
# text_pre = ViTokenizer.tokenize(text_pre)

# # Loại bỏ dấu câu và kí tự
# text_pre = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ_]', ' ', text_pre)


In [28]:
text_pre

'có mỗi iphone mà cả tuần này tinh_tế xào_nấu kho hâm lại chắc dùng tháng mới hết  '

In [31]:
from text_preprocessed import preprocessing_text
print(preprocessing_text("Có mỗi iphone  mà cả tuần này tinh tế xào nấu kho hâm lại chắc dùng 2 tháng mới hết 🤣"))

TypeError: 'NoneType' object is not subscriptable

In [21]:
error_indices = []

for index, row in enumerate(documents):
    try:
        documents.loc[index] = preprocessing_text(row)
    except Exception as e:
        print(f"Error processing row at index {index}: {e}")
        print(row)
        error_indices.append(index)

print("Rows with errors:", error_indices)


Error processing row at index 771: 'NoneType' object is not subscriptable
Có mỗi iphone  mà cả tuần này tinh tế xào nấu kho hâm lại chắc dùng 2 tháng mới hết 🤣
Error processing row at index 817: 'NoneType' object is not subscriptable
Bài này từ hồi 
iphone 11
iphone 12
iphone 13
iphone 14
iphone 15
…

“Hổng đc, hình em gái này chưa xinh, cũ rùi…”

Ái chà, copy paste mỏi nay rùi sửa thêm cẩn thận kẻo anh em lại mắng oan ta🤣
Error processing row at index 868: 'NoneType' object is not subscriptable
Suỵt nói nhỏ thôi, nó lại bảo tái định nghĩa chứ ko bảo đi sao chép đâu 🤣🤣🤣
Error processing row at index 888: 'NoneType' object is not subscriptable
muốn hết nóng thì đợi apple ra bản update giảm hiệu năng A17 Pro thôi 🤣
Error processing row at index 900: 'NoneType' object is not subscriptable
Định nghĩa 🤣🤣🤣 Văn của mấy thằng ifan khá hài hước
Error processing row at index 1187: 'NoneType' object is not subscriptable
Sống ở VN mà ko hiểu tiếng việt ah🤐. Lại vặn “không khác” và “ko khác nhiều” 

KeyboardInterrupt: 

In [20]:
documents

100      dài quá đọc được bài rồi em cuôn xuống và đi ra
101    kể hết các điểm trên con iphone đời_mới tích_c...
102    bài này viết chủ_đề lý_do mua từ tháng trước r...
103    vậy_mà iphone pro max bình_chọn tốt nhất chắc ...
104                                            thừa tiền
                             ...                        
496                   để mua iphone pro max thử tích_cực
497                            thớt chụp mấy tấm đẹp quá
498                                               cảm_ơn
499    cái x là cái vô_dụng nhất   ảnh chụp x ra nhìn...
500    nếu chụp trong điều_kiện thiếu sáng mà nét căn...
Name: Review, Length: 400, dtype: object

In [ ]:
clean_documents = [doc for doc in clean_documents if doc != '']


Feature Extraction = LSA = TruncatedSVD(TF-IDF)

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD,PCA

vectorizer = TfidfVectorizer(token_pattern = "\S+", min_df = 2)
vectors = vectorizer.fit_transform(clean_documents)

print("Tf-idf shape: " + str(vectors.shape))

svd = TruncatedSVD(n_components=100, n_iter=10, random_state=42)
svd_vectors = svd.fit_transform(vectors)

print("Document 1's Vector : ")
print(svd_vectors[0])

NameError: name 'clean_documents' is not defined

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD,PCA
vectorizer =  TfidfVectorizer(token_pattern="S+",min_df=2)
vectors = vectorizer.fit_transform(cleaned_documents)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [37]:
from pyvi import ViTokenizer

def vietnamese_tokenizer(text):
    return ViTokenizer.tokenize(text)

# Initialize TfidfVectorizer with Vietnamese tokenizer
vectorizer = TfidfVectorizer(tokenizer=vietnamese_tokenizer)

# Fit and transform the documents
vectors = vectorizer.fit_transform(cleaned_documents)

In [36]:
print("Tfidf shape:"+str(vectors.shape))
svd = TruncatedSVD(n_components=100,n_iter=10)
svd_vectors=svd.fit_transform(vectors)
print("Document 1's Vector : ")
print(svd_vectors[0])

Tfidf shape:(60285, 394)
Document 1's Vector : 
[ 9.75689395e-01 -3.66864101e-02  3.76155339e-02 -6.66627690e-03
  2.35498501e-02  3.35414139e-02  2.97596616e-02 -1.32784354e-02
  4.37651133e-02 -1.09132991e-01 -5.40689923e-03 -1.40232757e-02
 -7.15484564e-03  7.47907831e-03  2.95028372e-02  1.59393085e-02
  5.28207376e-02  1.50690770e-02  1.06979634e-02 -9.89425938e-03
  2.57207745e-04  4.06022106e-03  1.33053934e-02 -6.88813313e-03
 -1.35034359e-02  1.09001277e-02  4.11336451e-03  1.08588695e-02
  1.35921669e-02 -3.09220443e-02 -2.88219392e-02  3.38235836e-02
  1.74701702e-02  5.71909025e-03 -2.50218230e-02  2.21057832e-02
 -2.92006185e-02 -1.56376770e-02 -2.66019461e-03 -9.59141384e-03
  1.81684431e-02 -2.18200257e-02 -1.42638019e-02  1.27177427e-02
  3.50329398e-02  9.08934671e-03  1.48472162e-05 -7.05407752e-04
 -2.94085798e-02  2.51918361e-03 -1.08195184e-02  1.04738744e-02
  2.40394748e-02 -1.33588051e-03 -4.97608382e-04  7.02950472e-03
  1.22409105e-03  7.58772890e-04 -1.126395

In [39]:
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
def distance(vecs):
    vec1 = vecs[0]
    vecAll = vecs[1]
    Dis_matrix = pairwise_distances(vec1,vecAll,metric = 'cosine',n_jobs=1)
    Dis_matrix = Dis_matrix.astype(np.float16)
    return Dis_matrix

def chunks_vec(l, n):
    for i in range(0, l.shape[0], n):
        yield l[i:i + n]

vector_chunks = list(chunks_vec(svd_vectors,1000))
vector_chunks = [(i,svd_vectors) for i in vector_chunks]

pool = Pool(2)
Dis_matrix = pool.map(distance,vector_chunks)
Dis_matrix = np.vstack(Dis_matrix)
pool.terminate()

print('cosine distance between Document 1 and Document 2 : ', Dis_matrix[0][2])
print('cosine distance between Document 2 and Document 3 : ', Dis_matrix[2][3])

MemoryError: Unable to allocate 460. MiB for an array with shape (1000, 60285) and data type float64

In [ ]:
from copy import deepcopy

THRESHOLD = 0.5

graph = deepcopy(Dis_matrix)
graph[graph <= THRESHOLD] = 2
graph[graph != 2] = 0
graph[graph == 2] = 1
graph = graph.astype(np.int8)

In [ ]:
from scipy.sparse.csgraph import connected_components
res = connected_components(graph,directed=False)

In [ ]:
from collections import OrderedDict
cluster_labels = res[1]
num_cluster = res[0]
res_cluster = OrderedDict()

for i in range(0,len(cluster_labels)):
    if cluster_labels[i] in res_cluster: res_cluster[cluster_labels[i]].append(i)
    else: res_cluster[cluster_labels[i]] = [i]

res_cluster = [res_cluster[i] for i in range(0,num_cluster)]
res_cluster = [sorted(r) for r in res_cluster if len(r) > 1]
res_cluster.sort(key=len,reverse=True)

print("Number of cluster: ", len(res_cluster))
print("Number of clustered documents: ", len([j for i in res_cluster for j in i]))
print("Number of noise documents: ", len(documents) - len([j for i in res_cluster for j in i]))

In [ ]:
for i in range(0,len(res_cluster)):
    print("Cluster " + str(i))
    for idx in res_cluster[i]:
        print(documents[idx].split('\n')[0])
    print('\n')